In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import os
import sys
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D, Activation
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import LambdaCallback, TensorBoard
from tensorflow.keras import backend as K 
import tempfile 
import matplotlib.pyplot as plt 
from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline

In [3]:
# configuration parameters 
TRAIN_DATA_DIR = '/Users/sanchit/Documents/Projects/Datasets/fire_and_smoke_data/train/'
VALIDATION_DATA_DIR = '/Users/sanchit/Documents/Projects/Datasets/fire_and_smoke_data/val/'
TEST_DATA_DIR = '/Users/sanchit/Documents/Projects/Datasets/fire_and_smoke_data/test/'
MODEL_PATH = "./models/mobilenetv2_.h5"
TRAIN_SAMPLES = 2400
VALIDATION_SAMPLES = 490
TEST_SAMPLES = 490
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64
EPOCHS = 7
LABELS = ["fire", "nofire"]
LR_LOSS_PLOT_PATH = "./models/lr_loss_plot.png"
INIT_LR = 0.01 # this needs to be set as the max LR from the finding LR plot
FIND_LR_ENABLE = False # at first set it always to true to find out the best LRs

### data generators
create data generators for both training and validation sets and apply data augmentation only to training generator

In [4]:
# create training data generator and initialize it with data augmentation methods 
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                   horizontal_flip=True, 
                                   vertical_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   brightness_range=[0.6, 1.3],
                                   rotation_range=90, 
                                   zoom_range=0.15, fill_mode="reflect")

# and validation data generator
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=12345,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 2400 images belonging to 2 classes.
Found 490 images belonging to 2 classes.


### Create model 
we are using transfer learning for training our model, therefore, freeze the main layers of the model and attach a new custom F.C. classifier on the top of it.

In [5]:
# define a model 
def create_model():
    base_model = MobileNetV2(include_top=False, 
                              input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    
    for layer in base_model.layers[:]:
        layer.trainable = False
    
    input_layer = base_model.output 
    x = GlobalAveragePooling2D()(input_layer)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(NUM_CLASSES, name="last_dense")(x)
    output = Activation("softmax", name="last_softmax")(x)
    #output = Dense(NUM_CLASSES, activation='softmax')(x)
    
    # create the final model 
    model = Model(inputs = base_model.input, outputs = output)
    model.summary()
    return model 

### Build and compile model

In [6]:
# create model 
model = create_model()

# choose an optimize. Note for finding the best LR, it does not matter what lr is set 
# here! because during the best LR search, it will going to set its own LRs, however, at the end, 
# it will set back this original LRs. 
opt = tf.keras.optimizers.Adam(lr=INIT_LR)
#opt = tf.keras.optimizers.SGD(momentum=0.9)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

### Find the best learning rates range

In [7]:
class LearningRateFinder:
    """ Objective is to find a best learning rate by plotting various losses against a list of learning 
    rates which range from very high to very low. Optimal LR should lie somewhere inside of this range. 
    
    Starting and ending LRs which are chosen are too low (where network is unable to learn, thus a high 
    loss and too high (where loss is also high), respectively). Therefore, a good range of min and max LR 
    bounds should be somewhere inside of this range and finding that good range is the objective of this class. 
    
    At the end, entire network can be then trained by using either the correct LR (min LR) or min and max LRs 
    with a Cyclic LR scheduler. 
    
    Reference: https://www.pyimagesearch.com/2019/08/05/keras-learning-rate-finder/
    """
    def __init__(self, model, stopFactor=4, beta=0.98):
        """ initializes variables for finding the LRs 
            
            :param model: model for which LRs and losses are plotted and analyzed 
            :param stopFactor: stop factor when the LR becomes too large, then stop the model training automatically 
            :param beta: used for averaging the loss value 
            :param lrs: a list of tried LR values 
            :param losses: a list of tried loss values 
            :param avgLoss: average loss value over time 
            :param batchNum: current batch number 
            :param bestLoss: best loss (of course, lowest) found so far during training 
            :param lrMult: LR multiplication factor 
            :weightsFile: filename to save initial (original) weights of the model 
        """
        
        # store the model, stop factor, and beta value (for computing
        # a smoothed, average loss)
        self.model = model
        self.stopFactor = stopFactor
        self.beta = beta

        # initialize our list of learning rates and losses,
        # respectively
        self.lrs = []
        self.losses = []

        # initialize our learning rate multiplier, average loss, best
        # loss found thus far, current batch number, and weights file
        self.lrMult = 1
        self.avgLoss = 0
        self.bestLoss = 1e9
        self.batchNum = 0
        self.weightsFile = None

    def reset(self):
        # re-initialize all variables from our constructor
        self.lrs = []
        self.losses = []
        self.lrMult = 1
        self.avgLoss = 0
        self.bestLoss = 1e9
        self.batchNum = 0
        self.weightsFile = None

    def is_data_iter(self, data):
        # define the set of class types we will check for
        iterClasses = ["NumpyArrayIterator", "DirectoryIterator",
             "DataFrameIterator", "Iterator", "Sequence"]

        # return whether our data is an iterator
        return data.__class__.__name__ in iterClasses

    def on_batch_end(self, batch, logs):
        """ following are the steps/things which is happening in this function:
            - this function runs after every batch update 
            - recording of current loss values which is smoothen up first 
            - recording of best loss and updating of it if a new one has been found 
            - checking if the loss has grown too much, then stop the model training 
            - setting up of a new LR for the next model training with multiplying current 
              LR with the LR-multiplier at every batch update
        """
        
        
        # get the current LR from the model and save to a list of LRs which have been used already 
        lr = K.get_value(self.model.optimizer.lr)
        self.lrs.append(lr)

        # grab the loss at the end of this batch, increment the total
        # number of batches processed, compute the average average
        # loss, smooth it, and update the losses list with the
        # smoothed value
        l = logs["loss"] # NOTE: it contains the current loss of the training
        self.batchNum += 1
        self.avgLoss = (self.beta * self.avgLoss) + ((1 - self.beta) * l)
        smooth = self.avgLoss / (1 - (self.beta ** self.batchNum))
        self.losses.append(smooth)

        # compute the maximum loss stopping factor value
        stopLoss = self.stopFactor * self.bestLoss

        # check to see whether the loss has grown too large
        if self.batchNum > 1 and smooth > stopLoss:
            # stop returning and return from the method
            self.model.stop_training = True
            return

        # check to see if the best loss should be updated
        if self.batchNum == 1 or smooth < self.bestLoss:
            self.bestLoss = smooth

        # increase the learning rate ans set it as a new LR for the model training 
        lr *= self.lrMult # self.lrMult is found in the find() method
        K.set_value(self.model.optimizer.lr, lr)

    def find(self, trainData, startLR, endLR, epochs=None,
        stepsPerEpoch=None, batchSize=32, sampleSize=2048,
        classWeight=None, verbose=1):
        """ following are the steps/things which is happening in this function:
            - only using the training data for computing the loss, i.e., no split of it into test/val data 
            - loss log are transferred via LambdaCallback function to "on_batch_end()" fn. 
            - LR multiplier is computed once. And, it is a fixed (uniform) interval computed from endLR, startLR 
                over total numbers of batches 
            - Model's original weights and LR are temporarily saved and they get imported back again after 
              the plot for finding the best LR is done. 
        """
        
        # reset our class-specific variables
        self.reset()

        # determine if we are using a data generator or not
        useGen = self.is_data_iter(trainData)

        # if we're using a generator and the steps per epoch is not
        # supplied, raise an error
        if useGen and stepsPerEpoch is None:
            msg = "Using generator without supplying stepsPerEpoch"
            raise Exception(msg)

        # if we're not using a generator then our entire dataset must
        # already be in memory
        elif not useGen:
            # grab the number of samples in the training data and
            # then derive the number of steps per epoch
            numSamples = len(trainData[0])
            stepsPerEpoch = np.ceil(numSamples / float(batchSize))

        # if no number of training epochs are supplied, compute the
        # training epochs based on a default sample size
        if epochs is None:
            epochs = int(np.ceil(sampleSize / float(stepsPerEpoch)))

        # compute the total number of batch updates that will take
        # place while we are attempting to find a good starting
        # learning rate
        numBatchUpdates = epochs * stepsPerEpoch

        # derive the learning rate multiplier based on the ending
        # learning rate, starting learning rate, and total number of
        # batch updates
        self.lrMult = (endLR / startLR) ** (1.0 / numBatchUpdates)

        # save the model's original weights, so we can reset the weights when we are 
        # done finiding the optimal learning rates 
        self.weightsFile = tempfile.mkstemp()[1]
        self.model.save_weights(self.weightsFile)

        # grab the *original* learning rate (so we can reset it
        # later), and then set the *starting* learning rate
        origLR = K.get_value(self.model.optimizer.lr)
        K.set_value(self.model.optimizer.lr, startLR)

        # construct a callback that will be called at the end of each
        # batch, enabling us to increase our learning rate as training
        # progresses
        callback = LambdaCallback(on_batch_end=lambda batch, logs:
            self.on_batch_end(batch, logs))

        # check to see if we are using a data iterator
        if useGen:
            self.model.fit_generator(
                trainData,
                steps_per_epoch=stepsPerEpoch,
                epochs=epochs,
                class_weight=classWeight,
                verbose=verbose,
                callbacks=[callback])

        # otherwise, our entire training data is already in memory
        else:
            # train our model using Keras' fit method
            self.model.fit(
                trainData[0], trainData[1],
                batch_size=batchSize,
                epochs=epochs,
                class_weight=classWeight,
                callbacks=[callback],
                verbose=verbose)

        # finally, when we are done, set back the original model's weights and LR values 
        self.model.load_weights(self.weightsFile)
        K.set_value(self.model.optimizer.lr, origLR)

    def plot_loss(self, skipBegin=10, skipEnd=1, title=""):
        # grab the learning rate and losses values to plot
        lrs = self.lrs[skipBegin:-skipEnd]
        losses = self.losses[skipBegin:-skipEnd]

        # plot the learning rate vs. loss
        plt.plot(lrs, losses)
        plt.xscale("log")
        plt.xlabel("Learning Rate (Log Scale)")
        plt.ylabel("Loss")

        # if the title is not empty, add it to the plot
        if title != "":
            plt.title(title)

In [8]:
# find the best learning rates range only at the first run of this script. 
if FIND_LR_ENABLE:
    lrf = LearningRateFinder(model)
    # increasing learning rates at a regular interval from a very low to very high range. Then, compute 
    # the training loss. Objective here is to find a range of LRs where the loss varies the most. That means in those 
    # LRs model is the learning the most. 
    lrf.find(train_generator, 
            startLR=1e-10, 
            endLR=1e+1, 
            epochs=20,
            stepsPerEpoch=TRAIN_SAMPLES // BATCH_SIZE, 
            batchSize=BATCH_SIZE)

    # plot the loss for the various learning rates and save the
    # resulting plot to disk
    lrf.plot_loss()
    plt.savefig(LR_LOSS_PLOT_PATH)

    # exit the script so we can adjust our learning rates
    # in the config and then train the network for our full set of
    # epochs
    print("[INFO] learning rate finder complete")
    print("[INFO] examine plot and adjust learning rates before training")
    sys.exit(0)

### Train model 
After finding the min and max range of the best possible learning rates, train the whole model with the following 
possibilities: 
* If ADAM is used, then set the max of the range as the starting LR. As it will adapt it gradually towards a lower value. 
  
  
* If SGD is used, then max of the range as the starting LR and use either default decay or polynomial decay. But be careful that decay should not go beyond the min of the range. In other words, decay should gradually reach to the min of the range. 


* If SGD with Cyclic LR is used, then use [min_lr, max_lr] as the range. 

In [10]:
# define callbacks before starting the training
early_stop = EarlyStopping(monitor="val_loss", patience=5, mode="min", verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, min_lr=0.00001, verbose=1)
model_checkpoint = ModelCheckpoint(MODEL_PATH, monitor="val_acc", save_best_only=True, mode='max', verbose=1)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir=logdir, histogram_freq=1)
#callbacks = [reduce_lr, early_stop, model_checkpoint, tensorboard]
callbacks = [early_stop, model_checkpoint, tensorboard]

# perform model training 
H = model.fit_generator(generator=train_generator, 
                    steps_per_epoch=TRAIN_SAMPLES // BATCH_SIZE, 
                    epochs=EPOCHS,
                    validation_data=validation_generator,
                    validation_steps=VALIDATION_SAMPLES // BATCH_SIZE, 
                    callbacks=callbacks)

# launch TensorBoard
%tensorboard --logdir logs

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 37 steps, validate for 7 steps
Epoch 1/7
 5/37 [===>..........................] - ETA: 1:15 - loss: 0.2578 - acc: 0.8687

/anaconda3/envs/tf_env_2.1/lib/python3.7/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


36/37 [============================>.] - ETA: 2s - loss: 0.3031 - acc: 0.8609
Epoch 00001: val_acc improved from -inf to 0.82143, saving model to ./models/mobilenetv2_.h5
37/37 [==============================] - 96s 3s/step - loss: 0.3052 - acc: 0.8609 - val_loss: 0.3883 - val_acc: 0.8214
Epoch 2/7
36/37 [============================>.] - ETA: 2s - loss: 0.3047 - acc: 0.8719
Epoch 00002: val_acc did not improve from 0.82143
37/37 [==============================] - 107s 3s/step - loss: 0.3040 - acc: 0.8716 - val_loss: 0.4439 - val_acc: 0.8013
Epoch 3/7
36/37 [============================>.] - ETA: 2s - loss: 0.3181 - acc: 0.8644
Epoch 00003: val_acc did not improve from 0.82143
37/37 [==============================] - 101s 3s/step - loss: 0.3185 - acc: 0.8652 - val_loss: 0.7516 - val_acc: 0.7210
Epoch 4/7
36/37 [============================>.] - ETA: 2s - loss: 0.3613 - acc: 0.8495
Epoch 00004: val_acc improved from 0.82143 to 0.83036, saving model to ./models/mobilenetv2_.h5
37/37 [===

Reusing TensorBoard on port 6006 (pid 11144), started 0:37:47 ago. (Use '!kill 11144' to kill it.)

### Evaluate results 
Plot Confusion matrix and classification report on both validation and test samples

In [ ]:
def get_mismatches(y_true: list, y_pred: list, filenames: list):
    """ return a list of wrong predictions (or, mis-matches) done by the trained model 
        :param y_true: a list of true (or ground-truth) labels 
        :param y_pred: a list of predicted labels 
        :param filenames: a list of filenames corresponding to the true labels
    """
    mismatches_imgs = list(dict())
    for i in range(0, len(y_true)):
        if y_true[i] != y_pred[i]:
            print(f"True class: {LABELS[y_true[i]]}({y_true[i]}) and predicted as: {LABELS[y_pred[i]]}({y_pred[i]})")
            temp = dict()
            temp["filename"] = filenames[i]
            temp["true label"] = y_true[i]
            temp["pred label"] = y_pred[i]
            mismatches_imgs.append(temp)
        continue
    return mismatches_imgs

In [ ]:
# Plot confusion matrix, classification report and mis-matches information for the validation data
y_pred = model.predict_generator(validation_generator, steps=VALIDATION_SAMPLES // BATCH_SIZE + 1)
y_pred = np.argmax(y_pred, axis=1)
print("Confusion matrix:")
print(confusion_matrix(y_true=validation_generator.classes, y_pred=y_pred))
print("-"*50)
print("Classification report:")
print(classification_report(y_true=validation_generator.classes, y_pred=y_pred, target_names=LABELS))
print("Mismatches information:")
print(get_mismatches(validation_generator.classes, y_pred, validation_generator.filenames))

In [ ]:
# Similarly, plot confusion for the test dataset
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    TEST_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

In [ ]:
# Plot confusion matrix, classification report and mis-classification (or, mis-matches) information for the test data
y_pred = model.predict_generator(test_generator, steps=TEST_SAMPLES // BATCH_SIZE + 1)
y_pred = np.argmax(y_pred, axis=1)
print("Confusion matrix:")
print(confusion_matrix(y_true=test_generator.classes, y_pred=y_pred))
print("-"*50)
print("Classification report:")
print(classification_report(y_true=test_generator.classes, y_pred=y_pred, target_names=LABELS))
print("Mismatches information:")
print(get_mismatches(test_generator.classes, y_pred, validation_generator.filenames))